We will focus on the following workflow instance (for now, in the simplified problem, we will not use the Workflow or WorkflowSchedulingProblem classes, because we want to test the Unbalanced Penalization method https://arxiv.org/pdf/2211.13914.pdf), which is not included in QHyper. But I will leave it here just for reference (especially for observation of what data is in the time and cost matrices).

In [1]:
from QHyper.problems.workflow_scheduling import (
    Workflow,
    WorkflowSchedulingProblem,
)

tasks_file =  "workflows_data/workflows/3_tasks_3_machines_1_path.json"
machines_file = "workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json"
deadline = 13

workflow = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingProblem(workflow)

In [2]:
wsp.workflow.time_matrix

,MachineA,MachineB,MachineC
task1,6.0,2.0,4.0
task2,3.0,1.0,2.0
task3,12.0,4.0,8.0


In [3]:
wsp.workflow.cost_matrix

,MachineA,MachineB,MachineC
task1,6.0,8.0,8.0
task2,3.0,4.0,4.0
task3,12.0,16.0,16.0


Below is the list of the hyperparameters (weights) that can be changed. The results in here are taken from Optuna. I assumed that the weights hyper_b, hyper_c, hyper_d can be the same, since the constraint they represent has a very similar form.

In [4]:
hyper_params = {'cost_function_weight': 1.0, # weight for: cost function 
                'encoding_machine_1_weight': 37.89186033670198, # weight for: (x[0] + x[1] + x[2] - 1)**2
                'encoding_machine_2_weight': 37.89186033670198, # weight for: (x[3] + x[4] + x[5] - 1)**2
                'encoding_machine_3_weight': 37.89186033670198, # weight for: (x[6] + x[7] + x[8] - 1)**2
                'deadline_linear_form_weight': 15.536726433137282, # weight for: deadline constraint - linear form (-- this is from the unbalanced penalization approach)
                'deadline_quadratic_form_weight': 38.61604208771982} # weight for: deadline constraint - quadratic form

In [6]:
import numpy as np
import sympy
from sympy.core.expr import Expr

from QHyper.problems.base import Problem
from QHyper.util import Expression


class SimpleWorkflowProblem(Problem):

    
    def __init__(self) -> None:
        num_of_qubits = wsp.workflow.cost_matrix.shape[0] * wsp.workflow.cost_matrix.shape[1]
        self.variables = sympy.symbols(' '.join([f'x{i}' for i in range(num_of_qubits)]))                                  
        self._set_objective_function()
        self._set_constraints()
        
    def _set_objective_function(self) -> None:
        C_f = 6.0*self.variables[0] + 8.0*self.variables[1] + 8.0*self.variables[2] + 3.0*self.variables[3] + 4.0*self.variables[4] + 4.0*self.variables[5] + 12.0*self.variables[6] + 16.0*self.variables[7] + 16.0*self.variables[8]
        

        K_f4_linear = deadline - (6*self.variables[0] + 2*self.variables[1] + 4*self.variables[2] + 3*self.variables[3] +
                            1*self.variables[4] + 2*self.variables[5] + 12*self.variables[6] + 4*self.variables[7] + 8*self.variables[8])
                
        self.objective_function = Expression(hyper_params['cost_function_weight'] * C_f + hyper_params['deadline_linear_form_weight'] *  K_f4_linear)
        
    def _set_constraints(self):
        K_f1 = self.variables[0] + self.variables[1] + self.variables[2] - 1
        K_f2 = self.variables[3] + self.variables[4] + self.variables[5] - 1
        K_f3 = self.variables[6] + self.variables[7] + self.variables[8] - 1

        K_f4_squared = deadline - (6*self.variables[0] + 2*self.variables[1] + 4*self.variables[2] + 3*self.variables[3] +
                            1*self.variables[4] + 2*self.variables[5] + 12*self.variables[6] + 4*self.variables[7] + 8*self.variables[8])

            
        self.constraints = [Expression(K_f1), Expression(K_f2), Expression(K_f3), Expression(K_f4_squared)]
    
    def get_score(self, result, penalty=0):
        
        x = [int(val) for val in result]
    
        if (x[0] + x[1] + x[2] == 1 and 
            x[3] + x[4] + x[5] == 1 and 
            x[6] + x[7] + x[8] == 1 and 
            6*x[0] + 2*x[1] + 4*x[2] + 3*x[3] + 1*x[4] + 2*x[5] + 12*x[6] + 4*x[7] + 8*x[8] <= 13):
            
            return 6.0*x[0] + 8.0*x[1] + 8.0*x[2] + 3.0*x[3] + 4.0*x[4] + 4.0*x[5] + 12.0*x[6] + 16.0*x[7] + 16.0*x[8]
        
        return penalty

In [7]:
problem = SimpleWorkflowProblem()

In [9]:
print(f"Variables used to describe objective function"
      f" and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints (RHS == 0):")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective function and constraints: (x0, x1, x2, x3, x4, x5, x6, x7, x8)
Objective function: {('x0',): -87.2203585988237, ('x1',): -23.0734528662746, ('x2',): -54.1469057325491, ('x3',): -43.6101792994118, ('x4',): -11.5367264331373, ('x5',): -27.0734528662746, ('x6',): -174.440717197647, ('x7',): -46.1469057325491, ('x8',): -108.293811465098, (): 201.977443630785}
Constraints (RHS == 0):
    {('x0',): 1, ('x1',): 1, ('x2',): 1, (): -1}
    {('x3',): 1, ('x4',): 1, ('x5',): 1, (): -1}
    {('x6',): 1, ('x7',): 1, ('x8',): 1, (): -1}
    {('x0',): -6, ('x1',): -2, ('x2',): -4, ('x3',): -3, ('x4',): -1, ('x5',): -2, ('x6',): -12, ('x7',): -4, ('x8',): -8, (): 13}


# 3 Using QHyper

In [10]:
params_cofing = {
        'angles': [[0.1e-3]*5, [0.5]*5], # QAOA angles - first we have gammas (for the cost Hamiltonian), then we have betas (for the mixer)
        'hyper_args': [1, # do not change - this should be the weight for the 'cost function' but since in our cost function 
                          # we also have the deadline in the linear form (as of now it needs to be implemented this way due to QHyper limitations)
                          # the weight for the actual cost function is set there. THIS WILL NOT WORK WELL WITH HYPER-QAOA.
                          
                       hyper_params['encoding_machine_1_weight'], 
                       hyper_params['encoding_machine_2_weight'], 
                       hyper_params['encoding_machine_3_weight'], 
                       hyper_params['deadline_quadratic_form_weight']],
    }

In [27]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function
from QHyper.solvers.vqa.base import VQA
tester_config = {
    'pqc': {
        'type': 'wfqaoa',
        'layers': 5,
    }
}

tester = VQA(problem, config=tester_config)

In [28]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
        'mixer': 'pl_x_mixer',
    },
     'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
    },
}
vqa = VQA(problem, config=solver_config)

In [29]:
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

Best params: {'angles': array([[2.52134230e-02, 2.52620067e-02, 2.51954725e-02, 1.62153825e-04,
        8.35187555e-05],
       [4.98000694e-01, 4.98000842e-01, 5.23124730e-01, 5.23124734e-01,
        5.23124736e-01]]), 'hyper_args': array([ 1.        , 37.89186034, 37.89186034, 37.89186034, 38.61604209])}


In [31]:
best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyperoptimizer: {best_params['hyper_args']}")

Best results: (0.5260472806965945, {'010000100': 0.010181865150128075, '111101011': 0.010163915451403195, '000110010': 0.009837401710358625, '110010101': 0.009635306704334363, '000111000': 0.009220505714934562, '001110110': 0.009063404165712014, '000101010': 0.009029257821191248, '101011011': 0.008771937281414751, '000010000': 0.008754638699973855, '010110000': 0.0086893154565651, '001001011': 0.008392192473291002, '110001000': 0.008366273065458599, '110010100': 0.008197648917191722, '010000010': 0.00786115554506479, '101010110': 0.007683615156515984, '110110011': 0.007660618990199517, '000001010': 0.007528150247414125, '000001000': 0.007506103363149577, '110111011': 0.007427567438447483, '000000011': 0.007059795321458505, '011101110': 0.006711732254312408, '000000001': 0.006694316731335921, '001011110': 0.006680433091327662, '100101000': 0.006651677087718581, '010100110': 0.006527939800395996, '101000000': 0.006443205811935124, '001000010': 0.006244537932759756, '101110110': 0.0062356